<a href="https://colab.research.google.com/github/BU-Spark/ds-pit-un-syllabus/blob/Text-processing-topic-modeling/non_PIT_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytorch-lightning
!pip install transformers
!pip install sentencepiece
!pip install datasets
!pip install ml_things
!python -m pip uninstall matplotlib
!pip install matplotlib==3.1.3

# Install the appropriate libraries for use of the longformer model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 796 kB 16.8 MB/s 
     |████████████████████████████████| 136 kB 59.8 MB/s 
     |████████████████████████████████| 529 kB 19.5 MB/s 
     |████████████████████████████████| 87 kB 2.9 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=cbb5ab5b48302a83ac3c1c696eaa5ef4d820c1d1cb59cac2f24cbefc9d834f55
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 16.9 MB/s 
     |████████████████████████████████| 163 kB 70.2 MB/s 
     |████████████████████████████████| 7.6 MB 54.0 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████

Found existing installation: matplotlib 3.5.3
Uninstalling matplotlib-3.5.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3-py3.7-nspkg.pth
    /usr/local/lib/python3.7/dist-packages/matplotlib-3.5.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/matplotlib/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axes_grid1/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/axisartist/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/mplot3d/*
    /usr/local/lib/python3.7/dist-packages/mpl_toolkits/tests/*
    /usr/local/lib/python3.7/dist-packages/pylab.py
Proceed (y/n)? y
  Successfully uninstalled matplotlib-3.5.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.1 MB 23.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from ml_things import plot_dict, plot_confusion_matrix, fix_text
import pandas as pd
import numpy as np

In [ ]:
num_labels = 2
model_type = 'allenai/longformer-base-4096'

In [ ]:
# Downloading the longformer tokenizers

from transformers import LongformerTokenizer
checkpoint = model_type
tokenizer = LongformerTokenizer.from_pretrained(checkpoint, do_lower_case=True)

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

In [ ]:
# Check to see if the Google CoLab space has GPUs available to run the model

from transformers import AutoConfig, LongformerForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model_test = LongformerForSequenceClassification.from_pretrained(
    model_type, 
    num_labels = num_labels,   
    output_attentions = False,
    output_hidden_states = False,
)

# Tell pytorch to run this model on the GPU.
model_test.cuda()

Downloading:   0%|          | 0.00/597M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', '

RuntimeError: ignored

In [ ]:
# Importing the trained model from the Shared Google drive - link here to gain access: https://drive.google.com/drive/folders/1Pvk2cyHjWoPlYBGUBeFqY0lxi8zMdvzM?usp=sharing


import torch
model_test.load_state_dict(torch.load('/content/drive/Shareddrives/PIT Models/Model/longformer_preprocess2.cpkt',map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
# Function to predict the label from the model

def predicttestset(model, b_input_ids, b_input_mask):
  model.eval()

  # Tracking variables 
  predictions , true_labels = [], []
  total_eval_accuracy = 0
  total_eval_loss = 0

  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
    # Forward pass, calculate logit predictions.
    result = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask,
                    return_dict=True)

  # loss = result.loss
  logits = result.logits

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()

  # Store predictions and true labels
  predictions.append(logits)
  return predictions

In [ ]:
# Example technical non-PIT course text to test on the model

# Paste your text into the quotations here and then run the following cells

x_non_pit = " CAS CS 111: Introduction to Computer Science I Boston University, Fall 2022 Syllabus Description: The first course for computer science majors and anyone seeking a rigorous introduction. Develops computational problem-solving skills by programming in the Python language, and exposes students to variety of other topics from computer science and its applications. Carries MCS divisional credit in CAS. This course fulfills a single unit in each of the following BU Hub areas: Quantitative Reasoning II, Creativity/Innovation, Critical Thinking. No prerequisites. Important: CS 111 is a time-consuming and demanding course that is primarily intended for students who intend to take more advanced computer science courses. You should not take the course if you have other major time commitments, and you should not take it primarily for the purpose of fulfilling Hub areas. Other, less time- consuming options include CS 101, CS 103, CS 105, CS 108, and DS 100. Instructors A1/C1: See the course website for instructor, TA and CA office hours. Teaching Assistants (TAs) We are fortunate to have a number of undergraduate course assistants (CAs) as members of the course staff. They will be assisting you in the labs and holding office hours each week. See the course website for their names and contact info. Lectures and Labs lectures: section A1: MWF, 10:10-11:00 am section B1: MWF, 11:15 am-12:05 pm section C1: MWF, 12:20-1:10 pm section D1: MWF, 1:25-2:15 pm lab: a weekly session; see your schedule for the time and location Important: You must also be able to take the midterm exams, which will be held on two Wednesday evenings (10/19 and 11/16) from 6:30-7:45 p.m. We are not scheduled to meet on any other Wednesday evenings. COVID-19-Related Expectations To promote a safe learning environment, students must observe current University policies: https://www.bu.edu/shs/covid-19 CAS CS 111: Introduction to Computer Science I Syllabus, Fall 2022 page 2 of 5 Course Website: http://www.cs.bu.edu/courses/cs111 In addition, announcements and some course materials will be posted Blackboard. Requirements and Grading 1. Weekly problem sets and final project (30% of the final grade) 2. Exams: two midterm exams (30%) and a final exam (30%) 3. Participation (10%; see below) To pass the course, you must earn a passing grade on each of components 1 and 2. Collaboration Policy You are strongly encouraged to collaborate with one another in studying the lecture materials and preparing for the exams. Problem sets will include: • individual-only problems that you must complete on your own • pair-optional problems that you may complete alone or with a partner. For both types of problems, you may discuss ideas and approaches with others (provided that you acknowledge this in your solution), but such discussions should be kept at a high level and should not involve actual details of the code or of other types of answers. You must complete the actual solutions on your own (or, in the case of a pair-optional problem, with your partner if you choose to use one). Rules for working with a partner on pair-optional problems: • You may not work with more than one partner on a given assignment. (However, you are welcome to switch partners between assignments.) • You may not split up the work and complete it separately. • You must work together (at the same computer or via a Zoom meeting) for all problems completed as a pair, and your work must be a collaborative effort. • You and your partner must both submit the same solution to each problem that you did as a pair, and you must clearly indicate that you worked on the problem as a pair by putting your partner's name at the top of the file. Academic Misconduct We will assume that you understand BU’s Academic Conduct Code: http://www.bu.edu/academics/policies/academic-conduct-code You should also carefully review the CS department's page on academic integrity: http://www.bu.edu/cs/undergraduate/undergraduate-life/academic-integrity Prohibited behaviors include: • copying all or part of someone else's work, even if you subsequently modify it; this includes cases in which someone tells you what to write for your solution • viewing all or part of someone else's work (with the exception of work that you and your partner do together on a pair-optional problem) • showing all or part of your work to another student (with the exception of work that you and your partner do together on a pair-optional problem) • consulting solutions from past semesters, or those found online or in books • posting your work where others can view it (e.g., online) • receiving assistance from others or collaborating with others during an exam, or consulting materials except those that are explicitly allowed. (continued on next page) CAS CS 111: Introduction to Computer Science I Syllabus, Fall 2022 page 3 of 5 Incidents of academic misconduct will be reported to the Academic Conduct Committee (ACC). The ACC may suspend/expel students found guilty of misconduct. At a minimum, students who engage in misconduct will have their final grade reduced by one letter grade (e.g., from a B to a C). Course Materials • Required: CS 111 Coursepack. This contains all of the lecture notes for the course. More detail will be provided in class and in Lab 0. • Optional: CS for All by Alvarado, Dodds, Kuenning, and Libeskind-Hadas (Franklin Beedle, 2019). This book is not required. • Required: We will be using the Top Hat Pro platform. More detail will be provided in class. Other Policies Laptops: Students taking CS courses are expected to have a laptop capable of running a currently supported version of Microsoft Windows, Mac OS X, or Linux. See this page for more info: https://www.bu.edu/cs/undergraduate/undergraduate-life/laptops Late problem sets: Problem sets must be submitted by the date and time listed on the assignment (typically by 11:59 p.m.). There will be a 10% deduction for submissions up to 24 hours late. We will not accept any homework that is more than 24 hours late. Plan your time carefully, and don’t wait until the last minute so you will have time to ask questions and obtain assistance from the course staff. Pre-lecture preparation: To prepare for lecture, you will typically be required to watch one or two short videos and to complete an online quiz on Top Hat. Your work on these quizzes will not typically be graded for correctness, but it should demonstrate that you have adequately prepared for lecture. The pre-lecture quizzes must be submitted by the specified date and time; late submissions will not be accepted. The participation portion of your grade will be based on your completion of the pre- lecture quizzes and in-lecture questions on Top Hat, and on your consistent participation in the lab sessions. You will receive full credit for participation if you earn at least 85% of the points on Top Hat and if you participate in at least 85% of the lab sessions. If you earn x% of the Top Hat points or participate in x% of the lab sessions for a value of x that is less than 85, you will get x/85 of the possible points. Absences: The above participation policy is designed to allow for occasional absences due to illness or other special circumstances – including ones stemming from isolation for Covid. We will be recording the lectures and making the recordings available to everyone in the class. If you need to miss a lecture for any reason, you should simply watch the recording for that lecture as soon as possible after it is posted. In addition, you should keep up with the pre-lecture tasks and the current assignments. Please do not email your instructor for absences of this type. The final exam will replace your lowest problem-set grade if doing so helps your final grade. (The final-project grade cannot be replaced.) The final exam will also replace your lowest midterm-exam grade if doing so helps your final grade. Regardless of whether any such replacements occur, the final exam itself will always count for at least 30% of the final grade. CAS CS 111: Introduction to Computer Science I Syllabus, Fall 2022 page 4 of 5 The final grades are not curved. The performance of the class as a whole is taken into account in assigning letter grades, but this can only improve your grade, not harm it. Extensions and makeup exams will only be given in documented cases of serious illness or other emergencies. You cannot redo or complete extra work to improve your grade. Incompletes will not be given except in extraordinary circumstances. Schedule (tentative) week lecture dates topics, exams, assignments, and special dates 0 9/7, 9/9 No labs this week Course overview and introduction Getting started in Python Data types and expressions No labs this week 1 9/12, 9/14, 9/16 Strings and lists A first look at functions Making decisions (conditional execution) Problem Set 0 (all) due on 9/18 2 9/19, 9/21, 9/23 Functions (cont.) Local and global variables; the runtime stack Recursion 9/19: last day to add a class Problem Set 1, part I due on 9/22 Problem Set 1, part II due on 9/25 3 9/26, 9/28, 9/30 Recursion (cont.) List comprehensions Recursive design Problem Set 2, part I due on 9/29 Problem Set 2, part II due on 10/2 4 10/3, 10/5, 10/7 Lists of lists; encryption and decryption Algorithm design Representing information Problem Set 3, part I due on 10/6 Problem Set 3, part II due on 10/9 5 10/11, 10/12, 10/14 Binary addition Gates and circuits Minterm expansion No lecture on 10/10 (Indigenous Peoples' Day) Lecture on 10/11 (Mon. schedule) No labs this week 10/11: last day to drop without a 'W' Problem Set 4, part I due on 10/13 Problem Set 4, part II due on 10/16 "

In [ ]:
# Encodes the text in numerical form to test on the trained model

encoded_dict = tokenizer(
                        x_non_pit,                     # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 4096,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Setting the attention masks and the input IDs from the encoded dictionary, we want to use a CPU since the GPU was not available

device = torch.device("cpu")

input_ids = [encoded_dict['input_ids']]
    
# And its attention mask (simply differentiates padding from non-padding).
attention_masks = [encoded_dict['attention_mask']]

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0).to(device)
attention_masks = torch.cat(attention_masks, dim=0).to(device)

In [ ]:
# The prediction here will tell us if the text is more likely a PIT syllabus or a non-PIT syllabus.
# The numbers are logit numbers and the higher they are the more likely they associate that syllabus to be of that category.
# The left side is non-PIT association and the right is PIT, the right-side logit value is a positive 1.1 as opposed to the left side value that is
# -1.46 indicating that this example non-PIT technical syllabus is getting classified as a PIT syllabus.

prediction = predicttestset(model_test, input_ids, attention_masks)
prediction

[array([[-1.4568558,  1.1055523]], dtype=float32)]